In [ ]:
import numpy as np
import astropy.units as u
from astropy.constants import h, m_e, M_sun, c, sigma_T
from agnpy.emission_regions import Blob
from agnpy.targets import SSDisk, SphericalShellBLR, RingDustTorus
from agnpy.absorption import Absorption, sigma
from agnpy.compton import cos_psi
import matplotlib.pyplot as plt

In [ ]:
mec2 = m_e.to("erg", equivalencies=u.mass_energy())
# equivalency to transform frequencies to energies in electron rest mass units
epsilon_equivalency = [
    (u.Hz, u.Unit(""), lambda x: h.cgs * x / mec2, lambda x: x * mec2 / h.cgs)
]

In [ ]:
# define the blob
spectrum_norm = 1e47 * u.erg
parameters = {"p": 2.8, "gamma_min": 10, "gamma_max": 1e6}
spectrum_dict = {"type": "PowerLaw", "parameters": parameters}
R_b = 1e16 * u.cm
B = 0.56 * u.G
z = 0
delta_D = 40
Gamma = 40
blob = Blob(R_b, z, delta_D, Gamma, B, spectrum_norm, spectrum_dict)

In [ ]:
# disk parameters
M_BH = 1.2 * 1e9 * M_sun.cgs
L_disk = 2 * 1e46 * u.Unit("erg s-1")
eta = 1 / 12
R_in = 6
R_out = 200
disk = SSDisk(M_BH, L_disk, eta, R_in, R_out, R_g_units=True)

In [ ]:
# consider a fixed distance of the blob from the target fields
r = 1.1e16 * u.cm
r_tilde = (r / disk.R_g).to_value("")
l_tilde = np.logspace(0, 5) * r_tilde
print(l_tilde)

In [ ]:
def tau_disk(nu, l_tilde, disk):
    """assume mu_s = 1 -> cos psi = mu"""
    # define the dimensionless energy
    epsilon_1 = nu.to("", equivalencies=epsilon_equivalency)
    # each value of l, distance from the BH, defines a different range of
    # cosine integration, we have to break the integration as the array of
    # mu takes different values at each distance
    integrands_l = []
    for _l_tilde in l_tilde:
        mu = disk.mu_from_r_tilde(_l_tilde)
        epsilon = disk.epsilon_mu(mu, _l_tilde)
        integrand = (
            disk.phi_disk_mu(mu, _l_tilde) / (epsilon * mu * _l_tilde**3 * (mu**(-2) - 1)**(-3/2)) 
        )
        s = epsilon_1 * epsilon * (1 - mu) / 2
        cross_section = sigma(s).to_value("cm2")
        integrand *= (1 - mu) * cross_section
        integrands_l.append(np.trapz(integrand, mu, axis=0))
    prefactor = 3 * disk.L_disk / ((4 * np.pi)**2 * disk.R_g * m_e * c**3)
    integral = np.trapz(np.asarray(integrands_l), l_tilde, axis=0)
    tau = prefactor.to_value("cm-2") * integral
    return tau

In [ ]:
# compute our own tau
E = np.logspace(0, 5) * u.GeV
nu = E.to("Hz", equivalencies=u.spectral())
taus = np.asarray([tau_disk(_, l_tilde, disk) for _ in nu])

In [ ]:
# array of sampled taus
sampled_tau = np.loadtxt(
    f"agnpy/tests/sampled_taus/tau_disk_figure_14_finke_2016.txt", delimiter=",", comments="#",
)
E_ref = sampled_tau[:, 0] * u.GeV
taus_ref = sampled_tau[:, 1] 

In [ ]:
plt.loglog(E_ref, taus_ref, ls="-", marker="o", label="reference")
plt.loglog(E, taus, ls="--", marker=".", label="agnpy")
plt.legend()

In [ ]:
def tau_disk_finke(nu, l_tilde, disk):
    """use directly Eq. (63) in Finke 2016"""
    epsilon_1 = nu.to("", equivalencies=epsilon_equivalency)
    # axis 0: R_tilde
    # axis 1: l_tilde
    _R_tilde = np.reshape(disk.R_tilde, (disk.R_tilde.size, 1))
    _l_tilde = np.reshape(l_tilde, (1, l_tilde.size))
    _epsilon = disk.epsilon(_R_tilde)
    _phi_disk = disk.phi_disk(_R_tilde)
    _mu = np.sqrt(1 / (1 + _R_tilde**2 / _l_tilde**2))
    _s = epsilon_1 * _epsilon * (1 - _mu) / 2
    integrand = (
        _l_tilde**(-2) *
        _R_tilde**(-5/4) * 
        _phi_disk * 
        (1 + _R_tilde**2 / _l_tilde**2)**(-3/2) * 
        (sigma(_s) / sigma_T).to_value("") *
        (1 - _mu)
    )
    integral_R_tilde = np.trapz(integrand, disk.R_tilde, axis=0)
    integral_l_tilde = np.trapz(integral_R_tilde, l_tilde, axis=0)
    prefactor = 1e7 * disk.l_Edd**(3/4) * disk.M_8**(1/4) * disk.eta**(-3/4)
    return prefactor * integral_l_tilde 

In [ ]:
# compute our own tau
taus_finke = np.asarray([tau_disk_finke(_, l_tilde, disk) for _ in nu])

In [ ]:
plt.loglog(E_ref, taus_ref, ls="-", marker="o", label="reference")
plt.loglog(E, taus_finke, ls="--", marker=".", label="agnpy")
plt.legend()